In [ ]:
from IPython.display import HTML
HTML('''<style>html, body{overflow-y: visible !important} .CodeMirror{min-width:105% !important;} .rise-enabled .CodeMirror, .rise-enabled .output_subarea{font-size:140%; line-height:1.2; overflow: visible;} .output_subarea pre{width:110%}</style>''') # For slides

# Foundations of Data Mining: Assignment 1

Please complete all assignments in this notebook. You should submit this notebook, as well as a PDF version (See File > Download as).

In [ ]:
# Please fill in your names here
NAME_STUDENT_1 = "Jan-Niklas Schneider 1260421"
NAME_STUDENT_2 = "Georgiana Manolache 0876359"

In [ ]:
%matplotlib inline
from preamble import *
plt.rcParams['savefig.dpi'] = 100 # This controls the size of your figures
# Comment out and restart notebook if you only want the last output of each cell.
InteractiveShell.ast_node_interactivity = "all"

## MoneyBall (5 points, 1+2+1+1)
In the early 2000s, 2 baseball scouts completely changed the game of baseball by analysing the available data about baseball players and hiring the best ones.
The [MoneyBall dataset](https://www.openml.org/d/41021) contains this data (click the link for more details). The goal is to accurately predict the number of 'runs' each player can score. 

In [ ]:
moneyball = oml.datasets.get_dataset(41021) # Download MoneyBall data
# Get the predictors X and the target y
X, y, attribute_names = moneyball.get_data(target=moneyball.default_target_attribute, return_attribute_names=True)
# Describe the data with pandas, just to get an overview
ballframe = pd.DataFrame(X, columns=attribute_names)
ballframe.describe()

1 . Visually explore the data. Plot the distribution of each feature (e.g. histograms), as well as the target. Visualize the dependency of the target on each feature (use a 2d scatter plot). Is there anything that stands out? Is there something that you think might require special treatment?
- Feel free to create additional plots that help you understand the data
- Only visualize the data, you don't need to change it (yet)

Lets just have a look at some details of the data frame before actually visualize sutff

In [ ]:
print(attribute_names)


In [ ]:
print(ballframe.shape)

In [ ]:

ballframe.dtypes

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 15):
    print(ballframe.head(10))

Some NaN's are present in RankSeason and RankPlayoffs

In [ ]:
print('targets')
print(y)
print("nr of targets {0}".format(len(np.unique(y))))

In [ ]:
ballframe['Team'].value_counts()

Some teams are more often in data; probably due to having more games in play offs.
All data is encoded as floats.
What I want to do now is to see whats different between those teams with 47 appearances to the other ones.
[An Examination of the Moneyball Theory: A Baseball Statistical Analysis](http://thesportjournal.org/article/an-examination-of-the-moneyball-theory-a-baseball-statistical-analysis/)
this link might be interesting.

In [ ]:
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
import matplotlib.pyplot as plt

p1 = scatter_matrix(ballframe, alpha=0.2, figsize=(20, 16), diagonal='kde')

It looks like we have some features that are correlated and seem to draw decent linear function in a 2D environment. These features are SLG - OBP, BA - OBP, RA - OOBP, RA - OSLG, OOBP - OSLG. Lets do some additional plots for those features.

In [ ]:
columns = ['SLG', 'OBP', 'BA', 'RA', 'OOBP', 'OSLG']
p2 = scatter_matrix(ballframe[columns], alpha=0.2, figsize=(10, 10), diagonal='kde')

2 . Compare all linear regression algorithms that we covered in class (Linear Regression, Ridge, Lasso and ElasticNet), as well as kNN. Evaluate using cross-validation and the $R^2$ score, with the default parameters. Does scaling the data with StandardScaler help? Provide a concise but meaningful interpretation of the results.
- Preprocess the data as needed (e.g. are there nominal features that are not ordinal?). If you don't know how to proceed, remove the feature and continue.

In [ ]:
# nan_cols = ['RankSeason', 'RankPlayoffs']
ballframe = ballframe.fillna(0)

Simply filled the missing values for RankSeason and RankPlayoffs by 0 since they seem to be indepent of other variables as observed in the scatter matrices above. Additionally, the runs (target) should not be affected by the season or playoffs.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(ballframe, y, random_state=0)


In [ ]:
print("X_train: {0} ".format(X_train.shape))
print("X_test: {0} ".format(X_test.shape))
print("y_train: {0} ".format(y_train.shape))
print("y_test: {0} ".format(y_test.shape))
print("For scaled data the shape is the same.")

In the assignment it said to also consider also a StandardScaler. I will make use of if for the logistic regression. I am aware that scaling the input will have no noticable effect on the regression. Eventually accuracy will slightly differ because we do a second train/test-split on the day.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsClassifier

# initialize all regressors and knn
lr = LinearRegression()
rid = Ridge()
lasso = Lasso()
elastic = ElasticNet()
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
print(std_scaler)

In [ ]:
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(StandardScaler().fit_transform(ballframe), y, random_state=0)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

# regular data
lr_scores = cross_val_score(lr, X_train, y_train, cv=5)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print("Logistic regression test scores:")
print("cross val scores: {}".format(lr_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_lr)))
print("Average cv score: {:.2f}".format(lr_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(lr_scores)))

In [ ]:
# scaled data
lr_scores = cross_val_score(lr, X_train_scaled, y_train_scaled, cv=5)
lr.fit(X_train_scaled, y_train_scaled)
y_pred_lr = lr.predict(X_test_scaled)

print("Logistic regression test scores with scaled value:")
print("cross val scores: {}".format(lr_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_lr)))
print("Average cv score: {:.2f}".format(lr_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(lr_scores)))

In [ ]:
ridge_scores = cross_val_score(rid, X_train, y_train, cv=5)
rid.fit(X_train, y_train)
y_pred_ridge = rid.predict(X_test)

print("Ridge test scores:")
print("CV scores: {}".format(ridge_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_ridge)))
print("Average cv score: {:.2f}".format(ridge_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(ridge_scores)))

In [ ]:
lasso_scores = cross_val_score(lasso, X_train, y_train, cv=5)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

print("Lasso test scores:")
print("CV scores: {}".format(lasso_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_lasso)))
print("Average cv score: {:.2f}".format(lasso_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(lasso_scores)))

In [ ]:
elastic_scores = cross_val_score(elastic, X_train, y_train, cv=5)
elastic.fit(X_train, y_train)
y_pred_elastic = elastic.predict(X_test)

print("Elastic test scores:")
print("CV scores: {}".format(elastic_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_elastic)))
print("Average cv score: {:.2f}".format(elastic_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(elastic_scores)))

In [ ]:
knn_scores = cross_val_score(knn, X_train, y_train, cv=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("KNN test scores:")
print("cross val scores: {}".format(knn_scores))
print("R^2 score: {}".format(r2_score(y_test, y_pred_knn)))
print("Average cv score: {:.2f}".format(knn_scores.mean()))
print("Variance cv score: {:.4f}".format(np.var(knn_scores)))

3 . Do a default, shuffled train-test split and optimize the linear models for the degree of regularization ($alpha$) and choice of penalty (L1/L2). For Ridge and  Lasso, plot a curve showing the effect of the training and test set performance ($R^2$) while increasing the degree of regularization for different penalties. For ElasticNet, plot a heatmap $alpha \times l1\_ratio \rightarrow R^2$ using test set performance.
Report the optimal performance. Again, provide a concise but meaningful interpretation. What does the regularization do? Can you get better results?
- Think about how you get the L1/L2 loss. This is not a hyperparameter in regression.
- We've seen how to generate such heatmaps in Lecture 3.

The first part will look at the optimization of by tuning regularization and penalty by making use of gridsearch and compare to previous default results 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit


lasso_pipe = Pipeline([
    ('estimator', Lasso())
])
params_lasso = [{'estimator__alpha': [0.1,0.25,0.5,0.75,1]}]
# around 0.01 was doing better so we optimize around this value
lasso_grid = GridSearchCV(estimator=lasso_pipe, param_grid=params_lasso, cv=ShuffleSplit())

In [ ]:
lasso_grid.fit(X_train, y_train)

In [ ]:
print(lasso_grid.best_params_)
print(lasso_grid.best_score_)

In [ ]:
lasso_best = lasso_grid.best_estimator_
y_pred_lasso_best = lasso_best.predict(X_test)

print("Lasso test scores before optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_lasso)))

print("Lasso test scores after optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_lasso_best)))

In [ ]:
ridge_pipe = Pipeline([
    ('estimator', Ridge())
])
params_ridge = [{'estimator__alpha': [0.1,0.25,0.5,0.75,1]}]
ridge_grid = GridSearchCV(estimator=ridge_pipe, param_grid=params_ridge, cv=ShuffleSplit())
ridge_grid.fit(X_train, y_train)

In [ ]:
print(ridge_grid.best_params_)
print(ridge_grid.best_score_)

In [ ]:
ridge_best = ridge_grid.best_estimator_
y_pred_ridge_best = ridge_best.predict(X_test)

print("Ridge test scores before optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_ridge)))

print("Ridge test scores after optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_ridge_best)))

In [ ]:
elastic_pipe = Pipeline([
    ('estimator', ElasticNet())
])
params_elastic = [{'estimator__alpha': [0.1, 0.25,0.5,0.75,1],
                   'estimator__l1_ratio':[0,0.1,0.25,0.5,0.75,0.9,1]}]
elastic_grid = GridSearchCV(estimator=elastic_pipe, param_grid=params_elastic, cv=ShuffleSplit())
elastic_grid.fit(X_train, y_train)

In [ ]:
print(elastic_grid.best_params_)
print(elastic_grid.best_score_)
# l1_ratio tends to be 1, hence, we have a L1 penalty

In [ ]:
elastic_best = elastic_grid.best_estimator_
y_pred_elastic_best = elastic_best.predict(X_test)

print("ElasticNet test scores before optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_elastic)))

print("ElasticNet test scores after optimization:")
print("R^2 score: {}".format(r2_score(y_test, y_pred_elastic_best)))

Let's plot the learning curves in of Lasso/Ridge to see whether our optimized value actually optimize the model or if we under/overfit. For simplicity we will choose the highest scoring regularization values manually.

In [ ]:
def plot_learning_curve(alpha_val, title, train_sizes_, train_scores_, test_scores_):
    plt.figure()
    plt.title(title)
#     if ylim is not None:
#         plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_scores_mean = np.mean(train_scores_, axis=1)
    train_scores_std = np.std(train_scores_, axis=1)
    test_scores_mean = np.mean(test_scores_, axis=1)
    test_scores_std = np.std(test_scores_, axis=1)
    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=alpha_val, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=alpha_val, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="ShuffleSplit score")
    
    plt.legend(loc="best")
    plt.show()

In [ ]:
train_sizes = np.linspace(.1, 1.0, 5)

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(Lasso(alpha=0.003), ballframe, y, cv=ShuffleSplit(), train_sizes=train_sizes)
plot_learning_curve(0.003, "Lasso", train_sizes, train_scores, test_scores)

In [ ]:
# ridge
train_sizes, train_scores, test_scores = learning_curve(Ridge(alpha=0.001), ballframe, y, cv=ShuffleSplit(), train_sizes=train_sizes)
plot_learning_curve(0.003, "Ridge", train_sizes, train_scores, test_scores)

In [ ]:
heat_df = pd.DataFrame(elastic_grid.cv_results_)
print(heat_df.head(5))
heat_mean_scores = np.array(heat_df['mean_test_score']).reshape(5,7)

# plot the mean cross-validation scores
mglearn.tools.heatmap(heat_mean_scores, xlabel='L1_ratio', xticklabels=params_elastic[0]['estimator__l1_ratio'],
                      ylabel='Alpha', yticklabels=params_elastic[0]['estimator__alpha'], cmap="viridis")

4 . Visualize the coefficients of the optimized models. Do they agree on which features are
important? Compare the results with the feature importances returned by a RandomForest. Do it agree with the linear models? What would look for when scouting for a baseball player?

In [ ]:
elastic_coefs = elastic_grid.best_estimator_.named_steps['estimator'].coef_
ridge_coefs = ridge_grid.best_estimator_.named_steps['estimator'].coef_
lasso_coefs = lasso_grid.best_estimator_.named_steps['estimator'].coef_

In [ ]:
def normalize_df(df):
    return (df - df.mean()) / (df.max() - df.min())

In [ ]:
elastic_coefs_df = pd.DataFrame(elastic_coefs)
elastic_coefs_df.index = ballframe.columns
elastic_coefs_df = elastic_coefs_df.sort_values(by=0)

print('ElasticNet Coefficients:')
print(elastic_coefs_df)

In [ ]:
ridge_coefs_df = pd.DataFrame(ridge_coefs)
ridge_coefs_df.index = ballframe.columns
ridge_coefs_df = ridge_coefs_df.sort_values(by=0)

print(ridge_coefs_df)

In [ ]:
lasso_coefs_df = pd.DataFrame(lasso_coefs)
lasso_coefs_df.index = ballframe.columns
lasso_coefs_df = lasso_coefs_df.sort_values(by=0)
print(lasso_coefs_df)

In [ ]:
import seaborn as sns

sns.barplot(x=elastic_coefs_df[0], y=elastic_coefs_df.index, palette=sns.hls_palette(8, l=.3, s=.8)).set_title("ElasticNet")

In [ ]:
sns.barplot(x=lasso_coefs_df[0], y=lasso_coefs_df.index, palette=sns.hls_palette(8, l=.3, s=.8)).set_title("Lasso")

In [ ]:
sns.barplot(x=ridge_coefs_df[0], y=ridge_coefs_df.index, palette=sns.hls_palette(8, l=.3, s=.8)).set_title("Ridge")

From the plots we can clearly see which features are most important to these models. These features are clearly OBP and SLG are dominant in all models while in Ridge BA had a certain influence, too. Lets us have a look at the RandomForest whether it agrees with the regression models.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

In [ ]:
print(rfc.n_features_ )

print(rfc.feature_importances_)
print(sorted(zip(map(lambda x: round(x, 3), rfc.feature_importances_), ballframe.columns), reverse=True))

For the regression models most important features were most definitely SLG and OBP. If we compare the results of a RandomForestClassifier(RFC) with default parameters, we see that the RFC agrees with the most important feature, SLG. RFC, however, regards BA and RA higher. While RA was not present in the regression models, BA was observed to be a more important feature in Ridge. Another notable point in RFC is having surprisingly high importance of Year and Team which could be due to Teams having more impact in certain years such that RFC regards them more important than their actual value. 

When Scouting for players SLG is clearly the most significant value followed by OBP. Other values to consider are BA and RA.

## Nepalese character recognition (5 points, 1+2+2)
The [Devnagari-Script dataset](https://www.openml.org/d/40923) contains 92,000 images (32x32 pixels) of 46 characters from Devanagari script. Your goal is to learn to recognize the right letter given the image.

In [ ]:
devnagari = oml.datasets.get_dataset(40923) # Download Devnagari data
# Get the predictors X and the labels y
X, y = devnagari.get_data(target=devnagari.default_target_attribute); 
classes = devnagari.retrieve_class_labels(target_name='character') # This one takes a while, skip if not needed

In [ ]:
from random import randint
# Take some random examples, reshape to a 32x32 image and plot
fig, axes = plt.subplots(1, 5,  figsize=(10, 5))
for i in range(5):
    n = randint(0,90000)
    axes[i].imshow(X[n].reshape(32, 32), cmap=plt.cm.gray_r)
    axes[i].set_xlabel("Class: %s" % (classes[y[n]]))
plt.show();

1. Evaluate k-Nearest Neighbors, Logistic Regression and RandomForests with their default settings.
    - Take a stratified 10% subsample of the data.
    - Use the default train-test split and predictive accuracy. Is predictive accuracy a good scoring measure for this problem?
    - Try to build the same models on increasingly large samples of the dataset (e.g. 10%, 20%,...). Plot the training time and the predictive performance for each. Stop when the training time becomes prohibitively large (this will be different for different models). 

2 . Optimize the value for the number of neighbors $k$ (keep $k$ < 50) and the number of trees (keep $n\_estimators$ < 100) on the stratified 10% subsample.
- Use 10-fold crossvalidation and plot $k$ and $n\_estimators$ against the predictive accuracy. Which value of $k$, $n\_estimators$ should you pick?

3 . For the RandomForest, optimize both $n\_estimators$ and $max\_features$ at the same time on the entire dataset.
- Use a nested cross-validation and a random search over the possible values, and measure the accuracy. Explore how fine-grained this grid/random search can be, given your computational resources. What is the optimal performance you find?
- Hint: choose a nested cross-validation that is feasible. Don't use too many folds in the outer loop.
- Repeat the grid search and visualize the results as a plot (heatmap) $n\_estimators \times max\_features \rightarrow ACC$ with ACC visualized as the color of the data point. Try to make the grid as fine as possible. Interpret the results. Can you explain your observations? What did you learn about tuning RandomForests?

## 3. Understanding Ensembles (5 points (3+2))
Do a deeper analysis of how RandomForests and Gradient Boosting reduce their prediction error. We'll use the MAGIC telescope dataset (http://www.openml.org/d/1120). When high-energy particles hit the atmosphere, they produce chain reactions of other particles called 'showers', and you need to detect whether these are caused by gamma rays or cosmic rays.

In [ ]:
# Get the data
magic_data = oml.datasets.get_dataset(1120) # Download MAGIC Telescope data
X, y = magic_data.get_data(target=magic_data.default_target_attribute);

In [ ]:
# Quick visualization
X, y, attribute_names = magic_data.get_data(target=magic_data.default_target_attribute, return_attribute_names=True)
magic = pd.DataFrame(X, columns=attribute_names)
magic.plot(figsize=(20,10))
# Also plot the target: 1 = gamma, 0 = background
pd.DataFrame(y).plot(figsize=(20,1));

1 . Do a bias-variance analysis of both algorithms. For each, vary the number of trees on a log scale from 1 to 1024, and plot the bias error (squared), variance, and total error (in one plot per algorithm). Interpret the results. Which error is highest for small ensembles, and which reduced most by each algorithm as you use a larger ensemble? When are both algorithms under- or overfitting? Provide a detailed explanation of why random forests and gradient boosting behave this way.
- See lecture 3 for an example on how to do the bias-variance decomposition
- To save time, you can use a 10% stratified subsample in your initial experiments, but show the plots for the full dataset in your report.

2 . A _validation curve_ can help you understand when a model starts under- or overfitting. It plots both training and test set error as you change certain characteristics of your model, e.g. one or more hyperparameters. Build validation curves for gradient boosting, evaluated using AUROC, by varying the number of iterations between 1 and 500. In addition, use at least two values for the learning rate (e.g. 0.1 and 1), and tree depth (e.g. 1 and 4). This will yield at least 4 curves. Interpret the results and provide a clear explanation for the results. When is the model over- or underfitting? Discuss the effect of the different combinations learning rate and tree depth and provide a clear explanation.
- While scikit-learn has a `validation_curve` function, we'll use a modified version (below) that provides a lot more detail and can be used to study more than one hyperparameter. You can use a default train-test split.

In [ ]:
# Plots validation curves for every classifier in clfs. 
# Also indicates the optimal result by a vertical line
# Uses 1-AUROC, so lower is better
def validation_curve(clfs, X_test, y_test, X_train, y_train):
    for n,clf in enumerate(clfs):
        test_score = np.empty(len(clf.estimators_))
        train_score = np.empty(len(clf.estimators_))

        for i, pred in enumerate(clf.staged_decision_function(X_test)):
            test_score[i] = 1-roc_auc_score(y_test, pred)

        for i, pred in enumerate(clf.staged_decision_function(X_train)):
            train_score[i] = 1-roc_auc_score(y_train, pred)

        best_iter = np.argmin(test_score)
        learn = clf.get_params()['learning_rate']
        depth = clf.get_params()['max_depth']
        test_line = plt.plot(test_score,
                             label='learn=%.1f depth=%i (%.2f)'%(learn,depth,
                                                                 test_score[best_iter]))

        colour = test_line[-1].get_color()
        plt.plot(train_score, '--', color=colour)
        
        plt.xlabel("Number of boosting iterations")
        plt.ylabel("1 - area under ROC")
        plt.axvline(x=best_iter, color=colour)
        
    plt.legend(loc='best')